In [6]:
from functools import partial
import paddle
from paddlenlp.datasets import load_dataset
from paddlenlp.transformers import AutoModelForSequenceClassification, AutoTokenizer
from paddlenlp.trainer import Trainer, TrainingArguments


In [7]:
train_dataset = load_dataset("chnsenticorp", splits=["train"])
model = AutoModelForSequenceClassification.from_pretrained("ernie-1.0-base-zh", num_classes=len(train_dataset.label_list))
tokenizer = AutoTokenizer.from_pretrained("ernie-1.0-base-zh")

def convert_example(example, tokenizer):
    encoded_inputs = tokenizer(text=example["text"], max_seq_len=128, pad_to_max_seq_len=True)
    encoded_inputs["labels"] = int(example["label"])
    return encoded_inputs

train_dataset = train_dataset.map(partial(convert_example, tokenizer=tokenizer))

[2022-05-24 10:15:24,851] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.modeling.ErnieForSequenceClassification'> to load 'ernie-1.0-base-zh'.
[2022-05-24 10:15:24,855] [    INFO] - Already cached /home/ricardo/.paddlenlp/models/ernie-1.0-base-zh/ernie_v1_chn_base.pdparams
[2022-05-24 10:15:38,888] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load 'ernie-1.0-base-zh'.
[2022-05-24 10:15:38,890] [    INFO] - Already cached /home/ricardo/.paddlenlp/models/ernie-1.0-base-zh/vocab.txt


In [8]:
print(train_dataset[100])

{'input_ids': [1, 250, 371, 243, 850, 4, 160, 850, 36, 4, 898, 1143, 614, 356, 165, 201, 340, 88, 5, 12043, 196, 13, 131, 7, 218, 11, 560, 242, 817, 122, 4, 28, 232, 160, 850, 4, 335, 45, 850, 49, 4, 238, 559, 16, 675, 111, 100, 207, 358, 281, 12043, 785, 51, 10, 243, 100, 40, 850, 12043, 47, 218, 276, 505, 4, 169, 1741, 957, 4, 1266, 500, 1010, 28, 86, 269, 4, 1701, 500, 28, 1598, 486, 28, 61, 335, 86, 4, 982, 187, 34, 87, 9, 102, 871, 579, 1593, 87, 999, 183, 4, 201, 170, 75, 10, 28, 232, 160, 850, 5, 4, 466, 28, 370, 566, 1266, 500, 5, 269, 4, 118, 102, 311, 86, 15, 27, 269, 4, 244, 15, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
training_args = TrainingArguments('./output')
training_args.do_train = True
training_args.evaluation_strategy = 'steps'
# print(training_args)

In [4]:
class Custom_MSE(paddle.nn.Layer):
    def __init__(self):
        super(Custom_MSE, self).__init__()

    def forward(self, predictions, target):
        square_difference = paddle.square(predictions - target)
        loss_value = paddle.mean(square_difference)
        return loss_value
    
    # def __call__(self, predictions, target):
    #   square_difference = paddle.square(y_predictions - target)
    #   loss_value = paddle.mean(square_difference)
    #   return loss_value

In [5]:
# trainer = Trainer(
#     model=model,
#     criterion=paddle.nn.loss.CrossEntropyLoss(),
#     args=training_args,
#     train_dataset=train_dataset if training_args.do_train else None,
#     tokenizer=tokenizer)

# if training_args.do_train:
#     train_result = trainer.train()
#     metrics = train_result.metrics
#     trainer.save_model()
#     trainer.log_metrics("train", metrics)
#     trainer.save_state()